In [6]:
# Input the path of config file, the dir containing all frames, and the txt file of points coordinates

config_path = "/Users/binxu/Documents/Carlos Lab/MacaqueFace/MacaqueFace-Binxu-2018-11-17/config.yaml"
img_dir = "/Users/binxu/Documents/Carlos Lab/MacaqueFace/Faces/" #"/Users/binxu/Documents/Carlos Lab/MacaqueFace/MacaqueFace-Binxu-2018-11-17/labeled-data/2018-11-16_153501_069/" # "/Users/binxu/Documents/Carlos Lab/MacaqueFace/Faces/"
ptsdata_dir = "/Users/binxu/Documents/Carlos Lab/MacaqueFace/Faces/Faces.txt"

In [7]:
import pandas as pd
import yaml
import os

with open(config_path, 'r') as stream:
    data_loaded = yaml.load(stream)
scorer = [ data_loaded['scorer'] ] # Note add [ ] for single string
bodyparts = data_loaded['bodyparts']

n_pts = len(bodyparts)
tmp_labels = [[0] * n_pts *2 , [ i//2 for i in range(2*n_pts)], [0, 1] *n_pts]
idx = pd.MultiIndex(levels=[scorer, bodyparts, ['x','y']], labels=tmp_labels, names=['scorer', 'bodyparts', 'coords'])

fn_list = os.listdir(img_dir)
fn_list_full = [img_dir+fn for fn in fn_list if ".png" in fn]
fn_list_full.sort()

datatable = pd.DataFrame(index=fn_list_full,columns=idx,dtype=float)

ptsdataframe = pd.read_table(ptsdata_dir, header=0, delimiter="\s+") 

assert ptsdataframe.slice.max() == len(fn_list_full) , "Annotated frame number different from total frame number" # png file list length is the same as pts data 

name = scorer[0]
for idx in ptsdataframe.index:
    tmp = ptsdataframe.xs(idx)
    row_j = tmp.slice
    part_name = bodyparts[tmp.color]
    datatable[name][part_name]['x'].iloc[row_j-1] = tmp['x']
    datatable[name][part_name]['y'].iloc[row_j-1] = tmp['y']

datatable.to_csv(os.path.join(img_dir, "CollectedData_" + name + ".csv") ) 
datatable.to_hdf(os.path.join(img_dir, "CollectedData_" + name + ".h5") ,'df_with_missing', format='table', mode='w') 